In [1]:
import requests
import hmac
import json
import base64
import hashlib
import pandas as pd
import datetime, time
from time import sleep

In [2]:
base_url = "https://api.sandbox.gemini.com"
endpoint = "/v1/order/new"
url = base_url + endpoint

In [3]:
gemini_api_key = "master-2rJeGEcrQbx6OKXvSvLR"
gemini_api_secret = "CZTk6cmFZgVPSnoHij22LZNFZ3S".encode()

In [4]:
while True:
    t = datetime.datetime.now()
    payload_nonce =  str(int(time.mktime(t.timetuple())*1000))
    try:
        response = requests.get('https://api.gemini.com/v1/pubticker/btcusd')
        btc_data_old = response.json()
        btc_data_old_price = float(btc_data_old['ask'])
        print('Price of btc is ',btc_data_old_price, ' at {}'.format(pd.Timestamp.now()))
    except Exception as e:
        print('Error obtaining BTC price.')
    
    sleep(300)
    
    try:
        response = requests.get('https://api.gemini.com/v1/pubticker/btcusd')
        btc_data_new = response.json()
        btc_data_new_price = float(btc_data_new['ask'])
        print('Price of btc is ',btc_data_new_price, ' at {}'.format(pd.Timestamp.now()))
    except Exception as e:
        print('Error obtaining BTC price.')
        
    percent = int(((btc_data_new_price - btc_data_old_price) * 100) / btc_data_old_price)
    
    if percent != 5:
        print('The trade requirement was not satisfied. The percentage move is at ',percent)
        sleep(0.1)
        continue
    else:
        if percent == 5:
            
            try:
                response = requests.get('https://api.gemini.com/v1/pubticker/ethusd')
                eth = response.json()
                eth_price = float(eth['ask'])
            except Exception as e:
                print('Error obtaining ETH price')
            
            payload = {
                "request": "/v1/order/new",
                "nonce": payload_nonce,
                "symbol": "ethusd",
                "amount": "1",
                "price": eth_price,
                "side": "buy",
                "type": "exchange limit",
                "account": "algotrading"
                }
        
            encoded_payload = json.dumps(payload).encode()
            b64 = base64.b64encode(encoded_payload)
            signature = hmac.new(gemini_api_secret, b64, hashlib.sha384).hexdigest()

            request_headers = { 'Content-Type': "text/plain",
                                'Content-Length': "0",
                                'X-GEMINI-APIKEY': gemini_api_key,
                                'X-GEMINI-PAYLOAD': b64,
                                'X-GEMINI-SIGNATURE': signature,
                                'Cache-Control': "no-cache" }
            try:
                response = requests.post(url,
                                        data=None,
                                        headers=request_headers)
            
                new_order = response.json()
                print(new_order)
                check_status = new_order['client_order_id']

                sleep(2)

                t = datetime.datetime.now()
                nonce =  str(int(time.mktime(t.timetuple())*1000))

                status = requests.get('https://api.gemini.com/v1/order/status/' + nonce + '/' + check_status)

                if status['is_true'] == 'True':
                    print(status)
                    print ('Order placed at {}'.format(pd.Timestamp.now()))
                    break


            except exception as e:
                print('Error placing order')
    
    

Price of btc is  41299.66  at 2021-01-08 14:21:23.851166
Price of btc is  41335.68  at 2021-01-08 14:21:25.387568
The trade requirement was not satisfied. The percentage move is at  0
Price of btc is  41335.63  at 2021-01-08 14:21:25.999048
Price of btc is  41335.57  at 2021-01-08 14:21:27.522183
The trade requirement was not satisfied. The percentage move is at  0
Price of btc is  41335.57  at 2021-01-08 14:21:28.166817
Price of btc is  41335.57  at 2021-01-08 14:21:29.688817
The trade requirement was not satisfied. The percentage move is at  0
Price of btc is  41335.57  at 2021-01-08 14:21:30.323862
Price of btc is  41335.15  at 2021-01-08 14:21:31.902523
The trade requirement was not satisfied. The percentage move is at  0
Price of btc is  41334.87  at 2021-01-08 14:21:32.537341
Price of btc is  41334.15  at 2021-01-08 14:21:34.056298
The trade requirement was not satisfied. The percentage move is at  0


KeyboardInterrupt: 